In [2]:
!pip install bert_score
!pip install rouge-score
!pip install evaluate
!pip install sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 KB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 54.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 93.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24955 sha256=c53e98eda3ff5c97da33b7d7419b376c71f0c461d79e54ccb381366a06aaca1e
  Stored in directory: /root/.cache/pip/wheels/9b/3d/39/09558097d3119ca0a4d462df68f22c6f3c1b345ac63a09b86e
Successfully built rouge-score
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import pandas as pd
from evaluate import load
from sentence_transformers import SentenceTransformer, util

model_sentence_transformers = SentenceTransformer('all-MiniLM-L6-v2')
rouge = load('rouge')
bertscore = load("bertscore")

## Load Test Data to read dialogues

In [34]:
test = pd.read_csv('TaskA-ValidationSet.csv')
gt = test['section_text'].tolist()
dia = test['dialogue'].str.lower().tolist()

# Reading Task-A Summaries generated from different models

In [5]:
import pickle
pred1 = pickle.load(open('val_section_prediction_m3.pkl', 'rb')) #gpt3 summary
pred2 = pickle.load(open('val_section_prediction_m6_1.pkl', 'rb')) #bart-large summary
pred3 = pickle.load(open('val_section_prediction_m6_2.pkl', 'rb')) #bart-large summary
pred4 = pickle.load(open('val_section_prediction_m4.pkl', 'rb')) #t5-base summary

# Ensemble Task-A summaries

In [31]:
import numpy as np

def similarity(s1, s2):
  embeddings1 = model_sentence_transformers.encode(s1, convert_to_tensor=True)
  embeddings2 = model_sentence_transformers.encode(s2, convert_to_tensor=True)
  cosine_scores = util.cos_sim(embeddings1, embeddings2)
  return float(cosine_scores[0][0])

def select_one(all_preds):
  C={}
  for i in range(len(all_preds)):
    sim=[]
    for j in range(len(all_preds)):
      if i!=j:
        sim.append(similarity(all_preds[i], all_preds[j]))
    C[all_preds[i]] = np.mean(sim)
  return C

def jaccard_sim(list1, list2):
    intersection = len(list(set(list1).intersection(list2)))
    union = (len(set(list1)) + len(set(list2))) - intersection
    return float(intersection) / union

def select_two(all_preds, m):
  score2 = {}
  for i in all_preds:
    score = jaccard_sim(i.lower().split(),m.split())
    score2[i] = score
  return score2

def blend_scores(score1, score2):
  final_scores = {}
  for k,v in score1.items():
    final_scores[k] = 0.5*v + 0.5*score2[k]
  resp = sorted(final_scores.items(), key=lambda item: item[1], reverse=True)
  return resp

def ensemble(pred1, pred2, pred3, pred4, dia):
  prediction = []
  for i,j,k,l,m in zip(pred1, pred2, pred3, pred4, dia):
    all_preds = [i,j,k,l]
    score1 = select_one(all_preds)
    score2 = select_two(all_preds, m.lower())
    final_scores = blend_scores(score1, score2)
    prediction.append(final_scores[0][0])
  return prediction

final_summary = ensemble(pred1, pred2, pred3, pred4, dia)

In [38]:
test_predictions_with_ids = []
for i,j in zip(test['ID'].tolist(), final_summary):
  test_predictions_with_ids.append([i,j])

In [39]:
test_df = pd.DataFrame(test_predictions_with_ids, columns = ['ID', 'SystemOutput2'])
test_df.to_csv('final_taskA_summarization_output.csv', index=False)